In [16]:
"""from google.colab import drive
drive.mount('/content/drive')"""

"from google.colab import drive\ndrive.mount('/content/drive')"

In [17]:
import pandas as pd
pd.set_option("display.max_rows", None)
import unicodedata
import gzip
import json

In [18]:
with gzip.open("./escp_shared_hackathon_dataset.json", "rt", encoding="utf-8") as f:
    data = json.load(f)

df = pd.DataFrame(data)

In [19]:
#!pip install deep-translator

In [20]:
df['brand_name'].nunique()

16965

In [21]:
import pandas as pd
import re
import unicodedata
from deep_translator import GoogleTranslator

# Function to clean brand names
def clean_brand_name(name):
    """
    Cleans brand names by:
    - Converting to lowercase
    - Removing accents
    - Removing special characters, dashes, and extra spaces
    - Keeping only alphanumeric characters
    """
    if pd.isna(name):  # Handle NaN values
        return None
    name = name.lower().strip()  # Convert to lowercase and strip spaces
    name = unicodedata.normalize('NFKD', name).encode('ASCII', 'ignore').decode('utf-8')  # Remove accents
    name = re.sub(r'[-"\'\.]', '', name)  # Remove dashes, quotes, apostrophes, and dots
    name = re.sub(r'[^a-zA-Z0-9\s]', ' ', name)  # Remove all special characters except spaces
    name = re.sub(r'\s+', ' ', name).strip()  # Normalize spaces
    return name

# Test the function
print(clean_brand_name("A.P.C"))  # Output: "apc"
print(clean_brand_name("L'Oréal"))  # Output: "loreal"
print(clean_brand_name("Chanel "))  # Output: "chanel"
print(clean_brand_name("Dolce&Gabbana"))  # Output: "dolcegabbana"

# Initialize the translator for Korean to English translation
translator = GoogleTranslator(source="ko", target="en")

# Ensure required columns exist before translation
if {'brand_name', 'language_code'}.issubset(df.columns):
    # Extract unique brand names in Korean
    ko_brands = df.loc[df['language_code'] == 'ko', 'brand_name'].dropna().unique()

    # Translate brand names from Korean to English
    translated_brands = {brand: translator.translate(brand) for brand in ko_brands}

    # Apply translation to the DataFrame
    df['translated_brand_name'] = df['brand_name'].map(translated_brands)

    print("Translated brand names added to DataFrame.")
else:
    print("Warning: Required columns for translation not found in DataFrame.")

# Ensure cleaning is applied to both original and translated brand names
if 'brand_name' in df.columns:
    df['clean_brand_name'] = df['brand_name'].apply(clean_brand_name)

if 'translated_brand_name' in df.columns:
    df['clean_translated_brand_name'] = df['translated_brand_name'].apply(clean_brand_name)

# Display the number of unique cleaned brand names
if 'clean_brand_name' in df.columns:
    print(f"Number of unique clean brand names: {df['clean_brand_name'].nunique()}")

if 'clean_translated_brand_name' in df.columns:
    print(f"Number of unique clean translated brand names: {df['clean_translated_brand_name'].nunique()}")


apc
loreal
chanel
dolce gabbana
Translated brand names added to DataFrame.
Number of unique clean brand names: 15578
Number of unique clean translated brand names: 199


In [22]:
import pandas as pd

# Read the file
with open("taxonomy-with-ids.en-US.txt", encoding="utf-8") as file:
    taxonomy_lines = file.readlines()  # Read all lines

# Process taxonomy to extract IDs and categories
taxonomy_data = []
for line in taxonomy_lines:
    parts = line.strip().split(" - ")  # Format: "ID - Category > Subcategory"
    if len(parts) == 2:
        category_id = int(parts[0])  # Convert Google Category ID to integer
        category_path = parts[1].split(" > ")  # Split into hierarchical categories
        taxonomy_data.append([category_id] + category_path)

# Convert to DataFrame
taxonomy_df = pd.DataFrame(taxonomy_data)

# Rename columns dynamically based on hierarchy depth
column_names = ["google_category_id"] + [f"category_level_{i}" for i in range(1, len(taxonomy_df.columns))]
taxonomy_df.columns = column_names

# Convert category ID to numeric
taxonomy_df["google_category_id"] = pd.to_numeric(taxonomy_df["google_category_id"], errors="coerce")

# Display first few rows
print(taxonomy_df.head(10))

# Save the file (Optional)
taxonomy_df.to_csv("google_taxonomy.csv", index=False)


   google_category_id        category_level_1 category_level_2  \
0                   1  Animals & Pet Supplies             None   
1                3237  Animals & Pet Supplies     Live Animals   
2                   2  Animals & Pet Supplies     Pet Supplies   
3                   3  Animals & Pet Supplies     Pet Supplies   
4                7385  Animals & Pet Supplies     Pet Supplies   
5              499954  Animals & Pet Supplies     Pet Supplies   
6                7386  Animals & Pet Supplies     Pet Supplies   
7                4989  Animals & Pet Supplies     Pet Supplies   
8                4990  Animals & Pet Supplies     Pet Supplies   
9                7398  Animals & Pet Supplies     Pet Supplies   

  category_level_3        category_level_4               category_level_5  \
0             None                    None                           None   
1             None                    None                           None   
2             None                    None

In [23]:

df = df.merge(taxonomy_df, on="google_category_id", how="left")

# Show the first few rows after merging
print(df.head())


   row_id brand_name language_code  google_category_id  row_weight  \
0       0       None            en                 691    0.505044   
1       1       None            ja                 325    0.632272   
2       2       None            nl                 176    0.492944   
3       3       None            pt                1033    0.582456   
4       4       100%            de              499845    0.355131   

  translated_brand_name clean_brand_name clean_translated_brand_name  \
0                   NaN             None                        None   
1                   NaN             None                        None   
2                   NaN             None                        None   
3                   NaN             None                        None   
4                   NaN              100                        None   

        category_level_1      category_level_2            category_level_3  \
0          Home & Garden         Lawn & Garden                   Gar

In [24]:
# Replace 'None' values with 'No subcategory' in all category columns
category_columns = [col for col in df.columns if col.startswith("category_level_")]
df[category_columns] = df[category_columns].replace('None', 'No subcategory')

# Keep all columns but only include category_level_1 from category levels
df_simplified = df.drop(columns=category_columns).join(df[['category_level_1']])

print(df_simplified.head(100))





    row_id                brand_name language_code  google_category_id  \
0        0                      None            en                 691   
1        1                      None            ja                 325   
2        2                      None            nl                 176   
3        3                      None            pt                1033   
4        4                      100%            de              499845   
5        5                    Adidas            en                3442   
6        6                  Aihogard            en                3809   
7        7                  Avantree            en                 264   
8        8                    BESDER            en                 390   
9        9             Bloomingville            de                4026   
10      10  Cliffs by White Mountain            en                 187   
11      11                Corneliani            en                5183   
12      12                       DGK  

In [25]:
import pandas as pd
import re
import unicodedata
from deep_translator import GoogleTranslator



# Step 1: Merge translated brand names into clean_brand_name if available
if 'clean_translated_brand_name' in df.columns:
    df["clean_brand_name"] = df["clean_brand_name"].fillna(df["clean_translated_brand_name"])

# Drop both 'translated_brand_name' and 'clean_translated_brand_name' after merging
df = df.drop(columns=["translated_brand_name", "clean_translated_brand_name"], errors='ignore')


# Display the first few rows where language_code is "ko" (Korean) for verification
df_ko = df[df["language_code"] == "ko"]


# Step 2: Initial Cleanup - Preserve Non-Latin Brand Names Before -1 Replacement
def clean_brand(value):
    if pd.isna(value) or str(value).strip() == "" or str(value).strip().lower() in {"none", "undefined", "unbranded", "empty"}:
        return -1  # Replace only true empty/unbranded values in clean_brand_name
    return value  # Keep valid brand names

# Apply initial cleanup function to clean_brand_name only
df['clean_brand_name'] = df['clean_brand_name'].apply(lambda x: -1 if pd.isna(x) or x in [None, "", "none", "undefined", "unbranded", "empty"] else x)
print(df.head())
# Save the final cleaned dataset
output_path = "./df_fully_cleaned.csv"
df.to_csv(output_path, index=False)

print(f"✅ Final cleaned file saved at: {output_path}")

df.to_csv("df_cleaned.csv", index=False)



   row_id brand_name language_code  google_category_id  row_weight  \
0       0       None            en                 691    0.505044   
1       1       None            ja                 325    0.632272   
2       2       None            nl                 176    0.492944   
3       3       None            pt                1033    0.582456   
4       4       100%            de              499845    0.355131   

  clean_brand_name       category_level_1      category_level_2  \
0               -1          Home & Garden         Lawn & Garden   
1               -1            Electronics             Computers   
2               -1  Apparel & Accessories  Clothing Accessories   
3               -1         Sporting Goods    Outdoor Recreation   
4              100         Sporting Goods    Outdoor Recreation   

             category_level_3       category_level_4 category_level_5  \
0                   Gardening        Disease Control             None   
1           Desktop Computers 